# Colab-specific code

In [1]:
%%capture
!pip install timm==0.9.16 captum==0.7.0 grad-cam==1.5.0

## Clone repository to cloned_repository folder and cd there.

In [2]:
!git clone https://github.com/mstaczek/cnn-vs-transformers-msc-thesis.git cloned_repository
import os
os.chdir("/content/cloned_repository/codes")
os.getcwd()

fatal: destination path 'cloned_repository' already exists and is not an empty directory.


'/content/cloned_repository/codes'

## Obtain data - Version 1 - Mount Google Drive, extract imagenette2 from there.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs('/content/drive/MyDrive/datasets/', exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import urllib.request

path_where_to_look_for_imagenette_tgz = "/content/drive/MyDrive/datasets/imagenette2.tgz"

# Check if imagenette2.tgz is in Google Drive
if os.path.isfile(path_where_to_look_for_imagenette_tgz):
    print("imagenette2.tgz exists. Skipping downloading")
else:
    print("imagenette2.tgz was not found at given path. Downloading")
    url = "https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz"
    output_filename = path_where_to_look_for_imagenette_tgz
    urllib.request.urlretrieve(url, output_filename)

# Extract tgz to cloned repo subfolder
os.system(f"tar -xvzf {path_where_to_look_for_imagenette_tgz} -C  /content/cloned_repository/datasets/")
!ls /content/cloned_repository/datasets/imagenette2

imagenette2.tgz exists. Skipping downloading
noisy_imagenette.csv  train  val


## Obtain data - Version 2 - just download imagenette2 and extract it.

Downloading:

In [9]:
# import urllib.request
# import os

# url = "https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz"
# output_filename = "/content/imagenette2.tgz"
# if not os.path.exists(output_filename):
#     urllib.request.urlretrieve(url, output_filename)
# else:
#     print("Dataset already downloaded")

# os.system(f"tar -xvzf {output_filename} -C  /content/cloned_repository/datasets/")
# !ls /content/cloned_repository/datasets/imagenette2

0

## Check if all imagenette2 files are present (should be 13395)

In [5]:
!find /content/cloned_repository/datasets/imagenette2 -type f | wc -l

13395


# Usage example

Get possible models etc.

In [6]:
from src import strings_to_classes_mappings

print("Possible datasets:", list(strings_to_classes_mappings.datasets_mapping.keys()))
print("Possible models:", list(strings_to_classes_mappings.models_mapping.keys()))
print("Possible explanations:", list(strings_to_classes_mappings.explanations_mapping.keys()))

Possible datasets: ['imagenette2']
Possible models: ['DeiT_S', 'DeiT_T', 'DenseNet121', 'EfficientNet_B3', 'EfficientNet_B4', 'ConvNeXtV2_Nano', 'PyramidViT_V2_B2', 'MobileNetV3', 'Swin_T', 'ResNet18', 'ResNet50', 'ResNeXt50', 'Res2Net50', 'ViT_B_32']
Possible explanations: ['GradCAM', 'KernelSHAP']


Set parameters of the analysis:

In [14]:
compute_explanations_parameters = {
    "number_of_batches_to_process" : 16,
    "batch_size" : 32,
    # "device" : "cpu", # for torch on cpu
    "device" : "cuda", # for torch on cuda gpu
    "shuffle" : True, # for dataloader
}

# example
# models_list = ['ResNet50', 'Swin_T']

# longer example
# models_list = ['DeiT_S', 'DeiT_T', 'DenseNet121', 'EfficientNet_B3', 'EfficientNet_B4',
#                'ConvNeXtV2_Nano', 'PyramidViT_V2_B2', 'MobileNetV3', 'Swin_T', 'ResNet18',
#                'ResNet50', 'ResNeXt50', 'Res2Net50', 'ViT_B_32']

# all implemented
models_list = list(strings_to_classes_mappings.models_mapping.keys())

In [ ]:
from datetime import datetime

def copy_explanations_to_drive(explanation_method, model_name):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    os.system(f"zip -r /content/{timestamp}_explanations_{explanation_method}_{model_name}.zip /content/cloned_repository/explanations/imagenette2/{explanation_method}/{model_name}")
    os.system(f"cp /content/{timestamp}_explanations_{explanation_method}_{model_name}.zip /content/drive/MyDrive/results/")
    print(f"Explanations for {explanation_method} and {model_name} copied to Drive: /content/drive/MyDrive/results/{timestamp}_explanations_{explanation_method}_{model_name}.zip")

## Test if saving works

In [ ]:
from src.compute_explanations import compute_explanations

compute_explanations_parameters_test = {
    "number_of_batches_to_process" : 2,
    "batch_size" : 2,
    "device" : "cuda",
    "shuffle" : True,
}

explanation_method = 'GradCAM'

for model in models_list[:2]:
    compute_explanations('imagenette2', model, explanation_method, **compute_explanations_parameters_test)
    copy_explanations_to_drive(explanation_method, model)

print("Checking if the files are in Drive...:\n")
!ls /content/drive/MyDrive/results/

## GradCAM

In [ ]:
from src.compute_explanations import compute_explanations

explanation_method = 'GradCAM'

for model in models_list:
    compute_explanations('imagenette2', model, explanation_method, **compute_explanations_parameters)
    copy_explanations_to_drive(explanation_method, model)

## KernelSHAP

In [ ]:
explanation_method = 'KernelSHAP'

for model in models_list:
    compute_explanations('imagenette2', model, explanation_method, **compute_explanations_parameters)
    copy_explanations_to_drive(explanation_method, model)

## Integrated Gradients

In [ ]:
explanation_method = 'IntegratedGradients'

for model in models_list:
    compute_explanations('imagenette2', model, explanation_method, **compute_explanations_parameters)
    copy_explanations_to_drive(explanation_method, model)